# Module 05_01: Introduction to Intel® Extension for Scikit-learn*


You are going to generate scores proving you communicated successfully and did sklearn computation on Intel GPU

![Assets/GalleryScores.png](Assets/GalleryScores.png)

<a id='Back_to_Sections'></a>

## Sections
- [Examine Optimized Functions in Sklearnex for GPU](#Examine-All-Optimized-Functions-in-Sklearnex)
- [Demonstrate usage of Intel® Extension for Scikit-learn with kmeans using patch_sklearn](#Demonstrate-usage-of-Intel-Extension-for-Scikit-learn-with-kmeans-using-patch_sklearn)
- [Compare timings of stock kmeans versus Intel® Extension for Scikit-learn kmeans](#Compare-timings-of-stock-kmeans-versus-Intel-Extension-for-Scikit-learn-kmeans)
- [Introduction to Daal4py](#Introduction-to-Daal4py)
- [oneAPI and GPU support in Intel® Extension for Scikit learn*](oneAPI-and-GPU-support-in-Intel-Extension-for-Scikit-learn*)
- [Plot DBSCAN results as computed on GPU](#Plot-DBSCAN-results-as-computed-on-GPU)


## Learning Objectives

* Apply Intel Extension for Scikit-learn to KNeighborsClassifier on Intel GPU




The following code demonstrates usage of Intel(R) Extension for Scikit-learn. Inspect code; there are no modifications necessary:
1. Inspect the following code cell and click Run (▶)to save the code to file.
2. Next, run (▶) the cell in the __Build and Run__ section following the code to compile and execute the code.

# Examine Which Scikit-learn* Are Optimized in Sklearnex

These following are the currently optimized functions optimized with Intel(r) Extensions for scikit learn

- [Back to Sections](#Back_to_Sections)

In [ ]:
import pandas as pd
sklearnex_gallery = pd.read_csv('../data/sklearnex_gallery_gpu.csv')
sklearnex_gallery

In [ ]:
def print_gallery_details(acroynm, column):
    print(sklearnex_gallery[sklearnex_gallery['Acronym'] == acroynm][column].tolist()[0])

details = ['What is it used for', 'Advantages', 'Disadvantages']
print_gallery_details('svc', 'Advantages')

## oneAPI and GPU support in Intel Extension for Scikit learn*
DPCTL is the python mechanism used to interface to SYCL device management. It is implemented via with sycl_context("xpu") blocks that redirect execution to a device of the selected type: GPU, CPU, or host. scikit-learn programs can be executed on GPU devices as well.

- [Back to Sections](#Back_to_Sections)

To patch your code with Intel CPU/GPU optimizations:

The following code demonstrates usage of simple DPPY code. Inspect code; there are no modifications necessary:
1. Inspect the following code cell and click Run (▶)to save the code to file.
2. Next, run (▶) the cell in the __Build and Run__ section following the code to compile and execute the code.

# Regarding when/how to cast to and from dpctl.tensors

This information bears repeating: to make sure the concept is clear.

Study the code sectons near the conversion to and from dptcl/Numpy

For all sklearnex alogorithms - it will be necessary to cast the X and/or y data passed as the parameter list to dpctl tensor in order for the GPU to access the data and performan the computation.

Examples:
- **x_device** = dpctl.tensor.from_numpy(**x**, usm_type = 'device', device = dpctl.SyclDevice("gpu"))
- **y_device** = dpctl.tensor.from_numpy(**y**, usm_type = 'device', device = dpctl.SyclDevice("gpu"))



Pay attention ot **return** types from:
- **fit** - many cases in scikit-learn, fit returns selfobject
- **fit_predict** - returns **ndarray** requires casting after the call on host (to_numpy)
- **predict** -  returns **ndarray** requires casting after the call on host (to_numpy)
- **fit_transform** - returns returns **ndarray** requires casting after the call on host (to_numpy)
- **tranform** - typically returns **ndarray** requires casting after the call on host (to_numpy)

Scikit-learn routines that potentially return ndarray type objects or which expect ndtype objects passed as a parameter will need to be cast to/from numpy from/to dpctl.tensor

To cast data being fed TO one of these routines:
- use dpctl.tensor.from_numpy() to conver from NumPy to dpctl tensor
- use dpctl.tensor.to_numpy() to convert from dpctl tensor to NumPy

Example: After a call to fit_predict:
- **catch_device** = estimator.fit_predict(**x_device**, **y_device**)
- **predictedHost** = dpctl.tensor.to_numpy(**catch_device**)


# Simple Compute Follows Data and Patching for kmeans

- [Back to Sections](#Back_to_Sections)

In [ ]:
%%writefile lab/scikit_kmeans.py

# Copyright 2021 Intel Corporation
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import numpy as np
from sklearnex import patch_sklearn
import dpctl
patch_sklearn()

X = np.array([[1,  2], [1,  4], [1,  0],
              [10, 2], [10, 4], [10, 0]])

# You need to re-import scikit-learn algorithms after the patch
from sklearn.cluster import KMeans

for d in dpctl.get_devices():
    gpu_available = False
    for d in dpctl.get_devices():
        if d.is_gpu:
            gpu_device = dpctl.select_gpu_device()
            gpu_available = True
        else:
            cpu_device = dpctl.select_cpu_device() 
if gpu_available:
    print("GPU targeted: ", gpu_device)
else:
    print("CPU targeted: ", cpu_device)

if gpu_available:
    # target a remote hosy CPU when submitted via q.sh or qsub -I
    x_device = dpctl.tensor.from_numpy(X, usm_type = 'device', device = dpctl.SyclDevice("gpu"))
    kmeans = KMeans(n_clusters=2, random_state=0).fit(X)
    print(f"kmeans.labels_ = {kmeans.labels_}")
else:
    print("GPU not found")

#### Build and Run
Select the cell below and click run ▶ to compile and execute the code:
- [Back to Sections](#Back_to_Sections)


In [ ]:
! chmod 755 q; chmod 755 run_kmeans_gpu.sh; if [ -x "$(command -v qsub)" ]; then ./q run_kmeans_gpu.sh; else ./run_kmeans_gpu.sh; fi

We observe that with scikit-learn-intelex patching you can:

Use your scikit-learn code for training and prediction with minimal changes (a couple of lines of code);
Fast execution training and prediction of scikit-learn models;
Get the same quality.

## Examine Compute Follows Data on Intel GPU with DBSCAN

In [ ]:
from sklearn.cluster import DBSCAN
#?DBSCAN.fit

In [ ]:
%%writefile lab/scikit_dbscan.py

# Copyright 2021 Intel Corporation
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

from sklearnex import patch_sklearn
patch_sklearn()

import pandas as pd
import numpy as np

from sklearn.cluster import DBSCAN
import dpctl

X = np.array([[1., 2.], [2., 2.], [2., 3.],
            [8., 7.], [8., 8.], [25., 80.]], dtype=np.float32)

for d in dpctl.get_devices():
    gpu_available = False
    for d in dpctl.get_devices():
        if d.is_gpu:
            gpu_device = dpctl.select_gpu_device()
            gpu_available = True
        else:
            cpu_device = dpctl.select_cpu_device() 
if gpu_available:
    print("GPU targeted: ", gpu_device)
else:
    print("CPU targeted: ", cpu_device)

if gpu_available:
    # target a remote hosy CPU when submitted via q.sh or qsub -I
    x_device = dpctl.tensor.from_numpy(X, usm_type = 'device', device = dpctl.SyclDevice("gpu"))
else:
    # target a remote hosy CPU when submitted via q.sh or qsub -I
    x_device = dpctl.tensor.from_numpy(X, usm_type = 'device', device = dpctl.SyclDevice("gpu"))

clustering_host = DBSCAN(eps=3, min_samples=2).fit(x_device)

print("DBSCAN.get_params: ",DBSCAN.get_params)
print('data type clustering_host.labels_', type(clustering_host.labels_))

# write meaningful cluster info to a CSV 
clustering_labels = pd.DataFrame(clustering_host.labels_)
clustering_labels.to_csv('../data/DBSCAN_Labels.csv', index=False )

clustering_components = pd.DataFrame(clustering_host.components_)
clustering_components.to_csv('../data/DBSCAN_Components.csv', index=False )

print("DBSCAN components: ", clustering_host.components_, "\nDBSCAN labels: ",clustering_host.labels_)

#### Build and Run
Select the cell below and click run ▶ to compile and execute the code:

In [ ]:
! chmod 755 q; chmod 755 run_scikit_dbscan.sh; if [ -x "$(command -v qsub)" ]; then ./q run_scikit_dbscan.sh; else ./run_scikit_dbscan.sh; fi

_If the Jupyter cells are not responsive, or if they error out when you compile the code samples, please restart the Jupyter Kernel: 
"Kernel->Restart Kernel and Clear All Outputs" and compile the code samples again_.

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

X = np.array([[1., 2.], [2., 2.], [2., 3.],
            [8., 7.], [8., 8.], [25., 80.]], dtype=np.float32)

labels = pd.read_csv('../data/DBSCAN_Components.csv')
labels = pd.read_csv('../data/DBSCAN_Labels.csv')

plt.title('DBSCAN Clusters - Fit')
plt.scatter(X[:, 0], X[:, 1], s=100, c=labels.iloc[:,0].tolist(), edgecolor="k")
plt.show()

# Use GPU to Fit, then use model to predict on host

We will test this method of **fitting on Intel GPU**, and **predicting on Intel CPU** over the following routines:
- **k_means_random**
- **linear_regression**
- **logistic_regression_lbfgs**
- **dbscan**


In [ ]:
%%writefile lab/dpctl_several_functions.py

#===============================================================================
# Copyright 2014-2021 Intel Corporation
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
#===============================================================================


import numpy as np
from sklearnex import patch_sklearn
patch_sklearn()

from sklearn.cluster import KMeans
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.cluster import DBSCAN

from sklearn.datasets import load_iris
import dpctl

def k_means_random(gpu_device):
    print("KMeans init='random'")
    X = np.array([[1., 2.], [1., 4.], [1., 0.],
                  [10., 2.], [10., 4.], [10., 0.]], dtype=np.float32)
    
    x_device = dpctl.tensor.from_numpy(X, usm_type = 'device', device = dpctl.SyclDevice("gpu"))
    kmeans = KMeans(n_clusters=2, random_state=0, init='random').fit(x_device)
    #kmeans = KMeans(n_clusters=2).fit(x_device)

    print("kmeans.labels_")
    print(kmeans.labels_)
    print("kmeans.predict([[0, 0], [12, 3]])")
    print(kmeans.predict(np.array([[0, 0], [12, 3]], dtype=np.float32)))
    print("kmeans.cluster_centers_")
    print(kmeans.cluster_centers_)


def linear_regression(gpu_device):
    print("LinearRegression")
    X = np.array([[1., 1.], [1., 2.], [2., 2.], [2., 3.]], dtype=np.float32)
    # y = 1 * x_0 + 2 * x_1 + 3
    y = np.dot(X, np.array([1, 2], dtype=np.float32)) + 3
           
    x_device = dpctl.tensor.from_numpy(X, usm_type = 'device', device = dpctl.SyclDevice("gpu"))
    y_device = dpctl.tensor.from_numpy(y, usm_type = 'device', device = dpctl.SyclDevice("gpu"))

    reg = LinearRegression().fit(x_device, y_device)
    print("reg.score(X, y)")
    print(reg.score(X, y))
    print("reg.coef_")
    print(reg.coef_)
    print("reg.intercept_")
    print(reg.intercept_)
    print("reg.predict(np.array([[3, 5]], dtype=np.float32))")
    print(reg.predict(np.array([[3, 5]], dtype=np.float32)))


def logistic_regression_lbfgs(gpu_device):
    print("LogisticRegression solver='lbfgs'")
    X, y = load_iris(return_X_y=True)
          
    x_device = dpctl.tensor.from_numpy(X, usm_type = 'device', device = dpctl.SyclDevice("gpu"))
    y_device = dpctl.tensor.from_numpy(y, usm_type = 'device', device = dpctl.SyclDevice("gpu"))

    clf = LogisticRegression(random_state=0, solver='lbfgs').fit(
        x_device,
        y_device)
    print("clf.predict(X[:2, :])")
    print(clf.predict(X[:2, :]))
    print("clf.predict_proba(X[:2, :])")
    print(clf.predict_proba(X[:2, :]))
    print("clf.score(X, y)")
    print(clf.score(X, y))

def dbscan(gpu_device):
    print("DBSCAN")
    X = np.array([[1., 2.], [2., 2.], [2., 3.],
                  [8., 7.], [8., 8.], [25., 80.]], dtype=np.float32)
    
    x_device = dpctl.tensor.from_numpy(X, usm_type = 'device', device = dpctl.SyclDevice("gpu"))    
    clustering = DBSCAN(eps=3, min_samples=2).fit(x_device)
    print("clustering.labels_")
    print(clustering.labels_)
    print("clustering")
    print(clustering)

if __name__ == "__main__":
    examples = [
        k_means_random, 
        dbscan,
        linear_regression,
        logistic_regression_lbfgs,
    ]

    for d in dpctl.get_devices():
        gpu_available = False
        for d in dpctl.get_devices():
            if d.is_gpu:
                gpu_device = dpctl.select_gpu_device()
                gpu_available = True
            else:
                cpu_device = dpctl.select_cpu_device() 
    if gpu_available:
        print("GPU targeted: ", gpu_device)
    else:
        print("CPU targeted: ", cpu_device)
        
    device = gpu_device
    for e in examples:
        print("*" * 80)
        try:
            e(device)
        except:
            print(e, " Failed")
        print("*" * 80)

    print('All looks good!')

#### Build and Run
Select the cell below and click run ▶ to compile and execute the code:

In [ ]:
! chmod 755 q; chmod 755 run_sklearn_several_on_GPU.sh; if [ -x "$(command -v qsub)" ]; then ./q run_sklearn_several_on_GPU.sh; else ./run_sklearn_several_on_GPU.sh; fi

# Summary
In this module you will have learned the following:
* How to target algorithms optimized by Intel Extensions for Scikit-learn* to run on Intel GPU using "Compute Follows Data"

# Notices & Disclaimers 

Intel technologies may require enabled hardware, software or service activation.
No product or component can be absolutely secure.

Your costs and results may vary.

© Intel Corporation. Intel, the Intel logo, and other Intel marks are trademarks of Intel Corporation or its subsidiaries. 
*Other names and brands may be claimed as the property of others.

